# 물류 최적 경로 탐색

## 데이터 불러오기

In [1]:
import pandas as pd

In [2]:
factories = pd.read_csv('tbl_factory.csv', index_col=0)
factories

,FCName,FCDemand,FCRegion
FCID,,,
FC00001,서울공장,28,북부
FC00002,수원공장,29,북부
FC00003,인천공장,31,북부
FC00004,대전공장,25,북부
FC00005,대구공장,21,남부
FC00006,광주공장,30,남부
FC00007,부산공장,25,남부
FC00008,울산공장,16,남부


In [3]:
warehouse = pd.read_csv('tbl_warehouse.csv', index_col=0)
warehouse

,WHName,WHSupply,WHRegion
WHID,,,
WH00001,판교창고,35,북부
WH00002,용인창고,41,북부
WH00003,수원창고,42,북부
WH00004,대전창고,60,남부
WH00005,대구창고,72,남부
WH00006,광주창고,65,남부


In [4]:
trans = pd.read_csv('tbl_transaction.csv', index_col=0)
trans

,TransactionDate,ToFC,FromWH,Quantity
TRID,,,,
0,2019-01-01 02:11:10,FC00004,WH00003,33
1,2019-01-01 06:12:42,FC00007,WH00006,19
2,2019-01-01 06:32:32,FC00006,WH00004,31
3,2019-01-01 07:17:06,FC00002,WH00003,18
4,2019-01-01 07:52:18,FC00001,WH00002,30
...,...,...,...,...
3995,2019-12-31 11:27:51,FC00007,WH00006,29
3996,2019-12-31 17:10:21,FC00002,WH00003,35
3997,2019-12-31 19:35:07,FC00003,WH00001,36


In [5]:
cost = pd.read_csv('rel_cost.csv', index_col=0)
cost

,FCID,WHID,Cost
RCostID,,,
1,FC00001,WH00001,0.4
2,FC00001,WH00002,0.8
3,FC00001,WH00003,1.5
4,FC00002,WH00001,0.9
5,FC00002,WH00002,0.8
6,FC00002,WH00003,1.6
7,FC00003,WH00001,1.2
8,FC00003,WH00002,1.3
9,FC00003,WH00003,1.5


## 데이터 병합

### trans(운송실적)을 기준으로 비용(cost) 데이터 병합

In [9]:
join_data = pd.merge(trans,cost , left_on=['ToFC', 'FromWH'], right_on=['FCID', 'WHID'], how='left')

In [10]:
join_data

,TransactionDate,ToFC,FromWH,Quantity,FCID,WHID,Cost
0,2019-01-01 02:11:10,FC00004,WH00003,33,FC00004,WH00003,1.1
1,2019-01-01 06:12:42,FC00007,WH00006,19,FC00007,WH00006,1.3
2,2019-01-01 06:32:32,FC00006,WH00004,31,FC00006,WH00004,0.9
3,2019-01-01 07:17:06,FC00002,WH00003,18,FC00002,WH00003,1.6
4,2019-01-01 07:52:18,FC00001,WH00002,30,FC00001,WH00002,0.8
...,...,...,...,...,...,...,...
3995,2019-12-31 11:27:51,FC00007,WH00006,29,FC00007,WH00006,1.3
3996,2019-12-31 17:10:21,FC00002,WH00003,35,FC00002,WH00003,1.6
3997,2019-12-31 19:35:07,FC00003,WH00001,36,FC00003,WH00001,1.2
3998,2019-12-31 21:45:07,FC00005,WH00005,34,FC00005,WH00005,0.3


### 조금 전 결합한 데이터(join_data)에서 공장데이터 병합

In [11]:
join_data = pd.merge(join_data,factories, left_on='ToFC', right_on ='FCID', how='left' )
join_data

,TransactionDate,ToFC,FromWH,Quantity,FCID,WHID,Cost,FCName,FCDemand,FCRegion
0,2019-01-01 02:11:10,FC00004,WH00003,33,FC00004,WH00003,1.1,대전공장,25,북부
1,2019-01-01 06:12:42,FC00007,WH00006,19,FC00007,WH00006,1.3,부산공장,25,남부
2,2019-01-01 06:32:32,FC00006,WH00004,31,FC00006,WH00004,0.9,광주공장,30,남부
3,2019-01-01 07:17:06,FC00002,WH00003,18,FC00002,WH00003,1.6,수원공장,29,북부
4,2019-01-01 07:52:18,FC00001,WH00002,30,FC00001,WH00002,0.8,서울공장,28,북부
...,...,...,...,...,...,...,...,...,...,...
3995,2019-12-31 11:27:51,FC00007,WH00006,29,FC00007,WH00006,1.3,부산공장,25,남부
3996,2019-12-31 17:10:21,FC00002,WH00003,35,FC00002,WH00003,1.6,수원공장,29,북부
3997,2019-12-31 19:35:07,FC00003,WH00001,36,FC00003,WH00001,1.2,인천공장,31,북부
3998,2019-12-31 21:45:07,FC00005,WH00005,34,FC00005,WH00005,0.3,대구공장,21,남부


### 창고 데이터도 병합

In [12]:
join_data = pd.merge(join_data,warehouse, left_on ='FromWH', right_on='WHID', how='left')
join_data

,TransactionDate,ToFC,FromWH,Quantity,FCID,WHID,Cost,FCName,FCDemand,FCRegion,WHName,WHSupply,WHRegion
0,2019-01-01 02:11:10,FC00004,WH00003,33,FC00004,WH00003,1.1,대전공장,25,북부,수원창고,42,북부
1,2019-01-01 06:12:42,FC00007,WH00006,19,FC00007,WH00006,1.3,부산공장,25,남부,광주창고,65,남부
2,2019-01-01 06:32:32,FC00006,WH00004,31,FC00006,WH00004,0.9,광주공장,30,남부,대전창고,60,남부
3,2019-01-01 07:17:06,FC00002,WH00003,18,FC00002,WH00003,1.6,수원공장,29,북부,수원창고,42,북부
4,2019-01-01 07:52:18,FC00001,WH00002,30,FC00001,WH00002,0.8,서울공장,28,북부,용인창고,41,북부
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,2019-12-31 11:27:51,FC00007,WH00006,29,FC00007,WH00006,1.3,부산공장,25,남부,광주창고,65,남부
3996,2019-12-31 17:10:21,FC00002,WH00003,35,FC00002,WH00003,1.6,수원공장,29,북부,수원창고,42,북부
3997,2019-12-31 19:35:07,FC00003,WH00001,36,FC00003,WH00001,1.2,인천공장,31,북부,판교창고,35,북부
3998,2019-12-31 21:45:07,FC00005,WH00005,34,FC00005,WH00005,0.3,대구공장,21,남부,대구창고,72,남부


In [14]:
# 필요한 컬럼만 추출
join_data = join_data[['TransactionDate', 'Quantity', 'Cost', 'ToFC', 'FCName','FCDemand', 'FromWH',  'WHName', 'WHSupply', 'WHRegion']]

In [15]:
join_data

,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FromWH,WHName,WHSupply,WHRegion
0,2019-01-01 02:11:10,33,1.1,FC00004,대전공장,25,WH00003,수원창고,42,북부
1,2019-01-01 06:12:42,19,1.3,FC00007,부산공장,25,WH00006,광주창고,65,남부
2,2019-01-01 06:32:32,31,0.9,FC00006,광주공장,30,WH00004,대전창고,60,남부
3,2019-01-01 07:17:06,18,1.6,FC00002,수원공장,29,WH00003,수원창고,42,북부
4,2019-01-01 07:52:18,30,0.8,FC00001,서울공장,28,WH00002,용인창고,41,북부
...,...,...,...,...,...,...,...,...,...,...
3995,2019-12-31 11:27:51,29,1.3,FC00007,부산공장,25,WH00006,광주창고,65,남부
3996,2019-12-31 17:10:21,35,1.6,FC00002,수원공장,29,WH00003,수원창고,42,북부
3997,2019-12-31 19:35:07,36,1.2,FC00003,인천공장,31,WH00001,판교창고,35,북부
3998,2019-12-31 21:45:07,34,0.3,FC00005,대구공장,21,WH00005,대구창고,72,남부


In [16]:
# 물류 데이터 저장
join_data.to_csv('logistics_master.csv', index = False)

## 데이터 탐색

### 북부 지역과 남부지역 비교

In [18]:
north_region =join_data.loc[join_data['WHRegion'] == '북부']
north_region

,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FromWH,WHName,WHSupply,WHRegion
0,2019-01-01 02:11:10,33,1.1,FC00004,대전공장,25,WH00003,수원창고,42,북부
3,2019-01-01 07:17:06,18,1.6,FC00002,수원공장,29,WH00003,수원창고,42,북부
4,2019-01-01 07:52:18,30,0.8,FC00001,서울공장,28,WH00002,용인창고,41,북부
7,2019-01-01 09:09:30,12,1.5,FC00001,서울공장,28,WH00003,수원창고,42,북부
8,2019-01-01 10:52:55,27,1.5,FC00003,인천공장,31,WH00003,수원창고,42,북부
...,...,...,...,...,...,...,...,...,...,...
3984,2019-12-30 12:29:42,22,0.9,FC00002,수원공장,29,WH00001,판교창고,35,북부
3987,2019-12-30 17:51:24,32,1.3,FC00003,인천공장,31,WH00002,용인창고,41,북부
3992,2019-12-31 01:50:16,33,0.8,FC00001,서울공장,28,WH00002,용인창고,41,북부
3996,2019-12-31 17:10:21,35,1.6,FC00002,수원공장,29,WH00003,수원창고,42,북부


In [19]:
south_region = join_data.loc[join_data['WHRegion'] == '남부']
south_region

,TransactionDate,Quantity,Cost,ToFC,FCName,FCDemand,FromWH,WHName,WHSupply,WHRegion
1,2019-01-01 06:12:42,19,1.3,FC00007,부산공장,25,WH00006,광주창고,65,남부
2,2019-01-01 06:32:32,31,0.9,FC00006,광주공장,30,WH00004,대전창고,60,남부
5,2019-01-01 08:56:09,31,0.3,FC00005,대구공장,21,WH00005,대구창고,72,남부
6,2019-01-01 09:00:15,33,0.7,FC00006,광주공장,30,WH00006,광주창고,65,남부
9,2019-01-01 14:12:51,21,0.7,FC00006,광주공장,30,WH00006,광주창고,65,남부
...,...,...,...,...,...,...,...,...,...,...
3993,2019-12-31 03:35:04,20,0.7,FC00006,광주공장,30,WH00006,광주창고,65,남부
3994,2019-12-31 07:05:56,11,0.8,FC00005,대구공장,21,WH00004,대전창고,60,남부
3995,2019-12-31 11:27:51,29,1.3,FC00007,부산공장,25,WH00006,광주창고,65,남부
3998,2019-12-31 21:45:07,34,0.3,FC00005,대구공장,21,WH00005,대구창고,72,남부


## 운송량과 비용확인

In [23]:
# 북부 지역과 남부지역의 총 비용 확인
print('북부 지역 총 비용:', str(north_region['Cost'].sum()) + '만원')
print('남부 지역 총 비용:', str(south_region['Cost'].sum()) + '만원')

북부 지역 총 비용: 2189.3만원
남부 지역 총 비용: 2062.0만원


In [30]:
# 북부 지역과 남부지역의 부폼 1개당 운송 비용 확인
north_cost = (north_region['Cost'].sum() / north_region['Quantity'].sum()) * 10000
print('북부 지역 부품 개당 운송비용',str(int(north_cost)) + '원')
south_cost = (south_region['Cost'].sum() / south_region['Quantity'].sum()) * 10000
print('남부 지역 부품 개당 운송비용',str(int(south_cost)) + '원')

북부 지역 부품 개당 운송비용 445원
남부 지역 부품 개당 운송비용 410원


In [32]:
# 평균운송 비용 확인
cost_chk= pd.merge(cost, factories, on='FCID', how='left')
cost_chk

,FCID,WHID,Cost,FCName,FCDemand,FCRegion
0,FC00001,WH00001,0.4,서울공장,28,북부
1,FC00001,WH00002,0.8,서울공장,28,북부
2,FC00001,WH00003,1.5,서울공장,28,북부
3,FC00002,WH00001,0.9,수원공장,29,북부
4,FC00002,WH00002,0.8,수원공장,29,북부
5,FC00002,WH00003,1.6,수원공장,29,북부
6,FC00003,WH00001,1.2,인천공장,31,북부
7,FC00003,WH00002,1.3,인천공장,31,북부
8,FC00003,WH00003,1.5,인천공장,31,북부
9,FC00004,WH00001,0.8,대전공장,25,북부


In [35]:
print('북부 지역 평균 운송 비용:',  cost_chk['Cost'].loc[cost_chk['FCRegion'] == "북부"].mean())
print('남부 지역 평균 운송 비용:',  cost_chk['Cost'].loc[cost_chk['FCRegion'] == "남부"].mean())

북부 지역 평균 운송 비용: 1.075
남부 지역 평균 운송 비용: 1.05


- 북부와 남부 지역의 공장들의 평균 운송 비용은 비슷함
- 그 결과 남부 지역의 공장들이 북부 지역 보다 효율 좋게 부품을 운송하고 있음